# Setup

In [ ]:
# installs
!pip3 install torch torchvision torchaudio
!pip3 install 'stable-baselines3[extra]' protobuf==3.20.\*
!pip3 install mss pyautogui pytesseract

In [ ]:
# imports
from mss import mss
import pyautogui
import cv2
import numpy as np
import pytesseract
from matplotlib import pyplot as plt
import time
from gym import Env
from gym.spaces import Box, Discrete

# Game Environment

In [ ]:
class WebGame(Env):
    def __init__(self):
        super().__init__()
        # Setup spaces
        self.observation_space = Box(low=0, high=255, shape=(1,60,150), dtype=np.uint8)
        self.action_space = Discrete(3)
        # Capture game frames
        self.cap = mss()
        # match to your window size
        self.game_location = {'top': 400, 'left': 0, 'width': 1400, 'height': 220}
        self.done_location = {'top': 405, 'left': 430, 'width': 660, 'height': 70}
        # store game over image here...will show how later
        self.template = cv2.imread('gameover.png',0)

        
        
    def step(self, action):
        action_map = {
            0:'space',
            1: 'down', 
            2: 'no_op'
        }
        if action !=2:
            pyautogui.press(action_map[action])

        done, done_cap = self.get_done() 
        observation = self.get_observation()
        if done:
            reward = -20
        else:
            reward = 1
        info = {}
        return observation, reward, done, info
        
    
    def reset(self):
        time.sleep(1)
        pyautogui.click(150, 150)
        pyautogui.press('space')
        return self.get_observation()
        
    def render(self):
        cv2.imshow('Game', self.current_frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            self.close()
         
    def close(self):
        cv2.destroyAllWindows()
        
    def get_observation(self):
        raw = np.array(self.cap.grab(self.game_location))[:,:,:3].astype(np.uint8)
        gray = cv2.cvtColor(raw, cv2.COLOR_BGR2GRAY)
        resized = cv2.resize(gray, (150,60))
        channel = np.reshape(resized, (1,60,150))
        return channel
    
    def get_done(self):
        done_cap = np.array(self.cap.grab(self.done_location))
        img = cv2.cvtColor(done_cap[:,:,:3], cv2.COLOR_RGB2GRAY)
        #plt.imshow(img)
        #plt.imshow(self.template)
        # on first run, uncomment so it saves 'game over' image, then comment out again
        # adjust path to your machine
        #cv2.imwrite('/Users/hunterstew/gameover.png', img)

        done = False
        res = cv2.matchTemplate(img,self.template,cv2.TM_CCOEFF_NORMED)[0][0]
        if res > .99:
            done = True
        return done, done_cap


In [ ]:
# test code to make sure its working
env = WebGame()
obs=env.get_observation()
#UNCOMMENT AND ADJUST self.game_location/done_location to your screen size
#plt.imshow(cv2.cvtColor(obs[0], cv2.COLOR_GRAY2BGR))

done, done_cap = env.get_done()
print(done)

for episode in range(1): 
    obs = env.reset()
    done = False  
    total_reward   = 0
    while not done: 
        obs, reward,  done, info =  env.step(env.action_space.sample())
        total_reward  +=    reward
    print('Total Reward for episode {} is {}'.format(episode, total_reward))       

# Helpers for saving trained models 

In [ ]:
# Import os for file path management
import os 
# Import Base Callback for saving models
from stable_baselines3.common.callbacks import BaseCallback
# Check Environment    
from stable_baselines3.common import env_checker

In [ ]:
env_checker.check_env(env)

In [ ]:
class TrainAndLoggingCallback(BaseCallback):

    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True

In [ ]:
CHECKPOINT_DIR = './trainv9/'
LOG_DIR = './logs/'

In [ ]:
callback = TrainAndLoggingCallback(check_freq=1000, save_path=CHECKPOINT_DIR)

# Train model

In [ ]:
from stable_baselines3 import DQN
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv, VecFrameStack

In [ ]:
env = WebGame()

In [ ]:
model = DQN('CnnPolicy', env, tensorboard_log=LOG_DIR, verbose=1, buffer_size=1200000, learning_starts=1000)

In [ ]:
model.learn(total_timesteps=1000000, callback=callback)

# Test model

In [ ]:
model = DQN.load('/Users/hunterstew/trainv8/best_model_325000.zip', env=env, print_system_info=True)

In [ ]:
for episode in range(5): 
    obs = env.reset()
    done = False
    total_reward = 0
    while not done: 
        action, _ = model.predict(obs)
        obs, reward, done, info = env.step(int(action))
        #plt.imshow(cv2.cvtColor(obs[0], cv2.COLOR_GRAY2BGR))
        total_reward += reward
    print('Total Reward for episode {} is {}'.format(episode, total_reward))